In [1]:
from config import *
import processing
import models
import torch.optim as optim
import torch.nn as nn
import numpy as np
import math
import torch

In [2]:
processing.preprocess_midi_files('F:\\GitHub\\dataset\\midi_dataset', 'F:\\GitHub\\dataset\\np_dataset')

 [F:\GitHub\dataset\midi_dataset\10cc\Dreadlock_Holiday.1.mid] [F:\GitHub\dataset\midi_dataset\10cc\Dreadlock_Holiday.2.mid] [F:\GitHub\dataset\midi_dataset\10cc\Dreadlock_Holiday.3.mid] [F:\GitHub\dataset\midi_dataset\10cc\Dreadlock_Holiday.4.mid] [F:\GitHub\dataset\midi_dataset\10cc\Dreadlock_Holiday.mid] [F:\GitHub\dataset\midi_dataset\10cc\Im_Not_In_Love.1.mid] [F:\GitHub\dataset\midi_dataset\10cc\Im_Not_In_Love.2.mid] [F:\GitHub\dataset\midi_dataset\10cc\Im_Not_In_Love.3.mid] [F:\GitHub\dataset\midi_dataset\10cc\Im_Not_In_Love.mid] [F:\GitHub\dataset\midi_dataset\10cc\The_Things_We_Do_for_Love.mid] [F:\GitHub\dataset\midi_dataset\2_Brothers_on_the_4th_Floor\Come_Take_My_Hand.mid] [F:\GitHub\dataset\midi_dataset\2_Brothers_on_the_4th_Floor\Dreams.1.mid] [F:\GitHub\dataset\midi_dataset\2_Brothers_on_the_4th_Floor\Dreams.mid] [F:\GitHub\dataset\midi_dataset\2_Brothers_on_the_4th_Floor\Let_Me_Be_Free.mid] [F:\GitHub\dataset\midi_dataset\2_Brothers_on_the_4th_Floor\Never_Alone.1.mid] [

C:\Users\Draco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


 [F:\GitHub\dataset\midi_dataset\883\Andra_tutto_bene_58_.1.mid] [F:\GitHub\dataset\midi_dataset\883\Andra_tutto_bene_58_.mid] [F:\GitHub\dataset\midi_dataset\883\Chiuditi_nel_cesso.1.mid] [F:\GitHub\dataset\midi_dataset\883\Chiuditi_nel_cesso.mid] [F:\GitHub\dataset\midi_dataset\883\Come_mai_feat._Fiorello_.1.mid] [F:\GitHub\dataset\midi_dataset\883\Come_mai_feat._Fiorello_.2.mid] [F:\GitHub\dataset\midi_dataset\883\Come_mai_feat._Fiorello_.mid] [F:\GitHub\dataset\midi_dataset\883\Con_un_deca.1.mid] [F:\GitHub\dataset\midi_dataset\883\Con_un_deca.mid] [F:\GitHub\dataset\midi_dataset\883\Cumuni.mid] [F:\GitHub\dataset\midi_dataset\883\Dimmi_perche_remix_.mid] [F:\GitHub\dataset\midi_dataset\883\Gli_anni_96_.1.mid] [F:\GitHub\dataset\midi_dataset\883\Gli_anni_96_.mid] [F:\GitHub\dataset\midi_dataset\883\Grazie_mille.1.mid] [F:\GitHub\dataset\midi_dataset\883\Grazie_mille.mid] [F:\GitHub\dataset\midi_dataset\883\Hanno_ucciso_luomo_ragno.1.mid] [F:\GitHub\dataset\midi_dataset\883\Hanno_uc

In [44]:
train_dataloader, test_dataloader = processing.get_train_test_dataloaders('F:\\GitHub\\dataset\\np_dataset')


TypeError: get_metadata() takes 0 positional arguments but 1 was given

In [42]:
def floor_to_nearest_10(number):
    return (number // 10) * 10

dataset = processing.SequenceDataset('F:\\GitHub\\dataset\\np_dataset')
metadata = processing.get_metadata()

# start_time = floor_to_nearest_10(min(metadata['artists']['Timeperiod'].values()))
# max_time = floor_to_nearest_10(max(metadata['artists']['Timeperiod'].values()))
# time_tokenized = {time : idx + 1 for idx, time in enumerate(range(start_time, max_time + 1, 10))}
genre_list = []
min_time, max_time = 1e9, 0
final_metadata = {}
for data in metadata['artists']:
    band = data['name']
    decade = floor_to_nearest_10(data['year_started'])
    min_time = min(min_time, decade)
    max_time = max(max_time, decade)
    genres = data['genres']
    for genre in genres:
        if genre not in genre_list:
            genre_list.append(genre)
    final_metadata[band] = {'decade' : decade,
                            'genres' : genres}

band_tokenized = {band : idx + 1 for idx, band in enumerate(final_metadata.keys())}
time_tokenized = {time : idx + 1 for idx, time in enumerate(range(min_time, max_time + 1, 10))}
genre_tokenized = {genre : idx + 1 for idx, genre in enumerate(genre_list)}

for band in final_metadata.keys():
    final_metadata[band]['band'] = band_tokenized[band]
    final_metadata[band]['decade'] = time_tokenized[final_metadata[band]['decade']]
    final_metadata[band]['genres'] = [genre_tokenized[genre] for genre in final_metadata[band]['genres']]


In [43]:
final_metadata

{'Queen': {'decade': 5, 'genres': [1, 2, 3], 'band': 1},
 'The_Beatles': {'decade': 4, 'genres': [1, 4, 5], 'band': 2},
 'Pooh': {'decade': 4, 'genres': [4, 1], 'band': 3},
 'Madonna': {'decade': 6, 'genres': [4, 6, 7], 'band': 4},
 'ABBA': {'decade': 5, 'genres': [4, 8], 'band': 5},
 'Metallica': {'decade': 6, 'genres': [9, 10], 'band': 6},
 'Genesis': {'decade': 4, 'genres': [2, 11], 'band': 7},
 'Led_Zeppelin': {'decade': 4, 'genres': [3, 12, 9], 'band': 8},
 'Jackson_Michael': {'decade': 5, 'genres': [4, 13, 14, 15], 'band': 9},
 'U2': {'decade': 5, 'genres': [1, 16, 17], 'band': 10},
 'John_Elton': {'decade': 4, 'genres': [4, 1], 'band': 11},
 'Battisti': {'decade': 4, 'genres': [4, 1], 'band': 12},
 'The_Rolling_Stones': {'decade': 4, 'genres': [1, 12], 'band': 13},
 'Eagles': {'decade': 5, 'genres': [1, 18], 'band': 14},
 'Celine_Dion': {'decade': 6, 'genres': [4, 19], 'band': 15},
 'Zucchero': {'decade': 5, 'genres': [4, 20, 1], 'band': 16},
 'Sting': {'decade': 5, 'genres': [1

In [ ]:
# Define the model, loss function, and optimizer
model = models.SimpleTransformer(VOCAB_SIZE, N_EMBD, N_HEAD, N_LAYER, FEEDFORWARD_DIM, BLOCK_SIZE, DROPOUT, DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataloader, test_dataloader = processing.get_train_test_dataloaders('F:\\GitHub\\dataset\\np_dataset')

# Training loop
num_epochs = EPOCHS
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (src, trg) in enumerate(train_dataloader):
        src, trg = src.to(DEVICE), trg.to(DEVICE)
        
        # Forward pass
        output = model(src)
        
        # Reshape output and target for loss calculation
        output = output.view(-1, VOCAB_SIZE)  # Flatten the output to [batch_size * seq_len, vocab_size]
        trg = trg.view(-1)  # Flatten the target to [batch_size * seq_len]
        
        # Compute loss
        loss = criterion(output, trg)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

    # Validation loop (optional)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():
        for src, trg in test_dataloader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            output = model(src)
            output = output.view(-1, VOCAB_SIZE)
            trg = trg.view(-1)
            val_loss += criterion(output, trg).item()
    
    avg_val_loss = val_loss / len(test_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

print("Training complete!")

Epoch [1/200], Average Loss: 6.9814
Epoch [1/200], Validation Loss: 6.6815
Epoch [2/200], Average Loss: 6.5833
Epoch [2/200], Validation Loss: 6.1422
Epoch [3/200], Average Loss: 6.3475
Epoch [3/200], Validation Loss: 6.3137
Epoch [4/200], Average Loss: 6.0033
Epoch [4/200], Validation Loss: 5.7499
Epoch [5/200], Average Loss: 5.8428
Epoch [5/200], Validation Loss: 5.8568
Epoch [6/200], Average Loss: 5.7911
Epoch [6/200], Validation Loss: 5.6117
Epoch [7/200], Average Loss: 5.5435
Epoch [7/200], Validation Loss: 5.2402
Epoch [8/200], Average Loss: 5.4650
Epoch [8/200], Validation Loss: 4.8031
Epoch [9/200], Average Loss: 5.1580
Epoch [9/200], Validation Loss: 5.8020
Epoch [10/200], Average Loss: 5.1629
Epoch [10/200], Validation Loss: 5.7064
Epoch [11/200], Average Loss: 5.1703
Epoch [11/200], Validation Loss: 5.8709
Epoch [12/200], Average Loss: 4.8042
Epoch [12/200], Validation Loss: 5.4733
Epoch [13/200], Average Loss: 5.0838
Epoch [13/200], Validation Loss: 4.3867
Epoch [14/200], A